In [21]:
DATA_DIR = '/d2/caches/kaggle-planet'
DATA_DIR_TRAIN = DATA_DIR + '/train-tif-v2'
DATA_DIR_TEST = DATA_DIR + '/test-tif-v2'
OUT_DIR = '/d2/caches/kaggle-planet/processed-images'

In [22]:
%run 'Lib.ipynb'

In [23]:
train_paths = [DATA_DIR_TRAIN + '/' + x for x in list_files(DATA_DIR_TRAIN)]
test_paths = [DATA_DIR_TEST + '/' + x for x in list_files(DATA_DIR_TEST)]
all_paths = train_paths + test_paths

In [24]:
len(all_paths)

101670

In [42]:
# load means & stds
mean_r = np.load('./out/mean_r_sub.npy')
std_r = np.load('./out/std_r_sub.npy', dtype=np.float64)
mean_g = np.load('./out/mean_g_sub.npy')
std_g = np.load('./out/std_g_sub.npy')
mean_b = np.load('./out/mean_b_sub.npy')
std_b = np.load('./out/std_b_sub.npy')
mean_nir = np.load('./out/mean_nir_sub.npy')
std_nir = np.load('./out/std_nir_sub.npy')

TypeError: load() got an unexpected keyword argument 'dtype'

In [33]:
for file_path in tqdm(all_paths):
    
    (f, _) = os.path.basename(file_path).split('.')
    d = read_geotiff(p)
    
    

  0%|          | 126/101670 [00:01<26:11, 64.62it/s]


KeyboardInterrupt: 

In [40]:
std_nir

array(1881.4609209678356)